In [18]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from joblib import Parallel, delayed

In [19]:

# Load dataset (Iris dataset for example)
import time


iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the models to train
def train_svm(X_train, y_train):
    start_time = time.time()

    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train)
    end_time = time.time()

    return svm_model, end_time - start_time

def train_knn(X_train, y_train):
    start_time = time.time()

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    end_time = time.time()

    return knn_model, end_time - start_time

def train_decision_tree(X_train, y_train):
    start_time = time.time()

    tree_model = DecisionTreeClassifier()
    tree_model.fit(X_train, y_train)
    end_time = time.time()

    return tree_model, end_time - start_time

In [36]:


n_jobs_proc=-1

# Use joblib to run all models in parallel by passing a list of delayed tasks
results = Parallel(n_jobs=n_jobs_proc)(
    [delayed(train_svm)(X_train, y_train),   # Train SVM
     delayed(train_knn)(X_train, y_train),   # Train KNN
     delayed(train_decision_tree)(X_train, y_train)]  # Train Decision Tree
)

# Unpack the trained models
(svm_model, svm_time1), (knn_model, knn_time1), (tree_model, tree_time1) = results



# Print accuracy and training time for each model
print(f" Training Time: {svm_time1:.4f} seconds")
print(f" Training Time: {knn_time1:.4f} seconds")
print(f" Training Time: {tree_time1:.4f} seconds")


 Training Time: 0.0050 seconds
 Training Time: 0.0010 seconds
 Training Time: 0.0030 seconds


## using DASK

In [21]:
import dask
from dask.distributed import Client ,LocalCluster
# Start Dask client with 4 workers (4 CPUs)
import dask.delayed



# Split dataset into training and test sets
features_train, features_test, labels_train, labels_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [32]:
cluster = LocalCluster(dashboard_address=':8888' , n_workers=-1 , threads_per_worker = -1)
cluster

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8888 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35986 instead
  warnings.warn(


Dashboard: http://127.0.0.1:35986/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35989,Workers: 0
Dashboard: http://127.0.0.1:35986/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [33]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:35986/status,
Dashboard: http://127.0.0.1:35986/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35989,Workers: 0
Dashboard: http://127.0.0.1:35986/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [34]:
# Define the models to train
def train_svm_model(features_train, labels_train):
    start = time.time()
    svm_clf = SVC(kernel='linear')
    svm_clf.fit(features_train, labels_train)
    end = time.time()
    return svm_clf, end - start

def train_knn_model(features_train, labels_train):
    start = time.time()
    knn_clf = KNeighborsClassifier(n_neighbors=5)
    knn_clf.fit(features_train, labels_train)
    end = time.time()
    return knn_clf, end - start

def train_tree_model(features_train, labels_train):
    start = time.time()
    tree_clf = DecisionTreeClassifier()
    tree_clf.fit(features_train, labels_train)
    end = time.time()
    return tree_clf, end - start

In [35]:

# Use Dask delayed to parallelize the training of models
svm_task_delayed = dask.delayed(train_svm_model)(X_train, y_train)
knn_task_delayed = dask.delayed(train_knn_model)(X_train, y_train)
tree_task_delayed = dask.delayed(train_tree_model)(X_train, y_train)

# Compute the tasks in parallel
trained_models = dask.compute(svm_task_delayed, knn_task_delayed, tree_task_delayed, scheduler='threads')

# Unpack the trained models and their training times
(svm_clf, svm_training_time), (knn_clf, knn_training_time), (tree_clf, tree_training_time) = trained_models

# Print training times for each model
print(f"SVM Training Time: {svm_training_time:.4f} seconds")
print(f"KNN Training Time: {knn_training_time:.4f} seconds")
print(f"Decision Tree Training Time: {tree_training_time:.4f} seconds")


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\base.py:1541: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


SVM Training Time: 0.0935 seconds
KNN Training Time: 0.0160 seconds
Decision Tree Training Time: 0.4969 seconds


In [37]:

# Shutdown Dask client
client.shutdown()

## instead of using CPU we try using GPU
### dask - cudaf

In [1]:
import time
import dask
import daal4py as d4p
from sklearn import datasets
from sklearn.model_selection import train_test_split
import dpctl  # Data Parallel Control for GPU management

# 1. Load the Iris dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Define functions to train each model using daal4py with GPU offloading


In [4]:

# Decision Tree Classifier with daal4py (GPU enabled)
def train_tree_model_gpu(X_train, y_train):
    start_time = time.time()
    with dpctl.SyclQueue("level_zero:gpu") as queue:
        tree_algo = d4p.decision_tree_classification_training(nClasses=3)
        tree_model = tree_algo.compute(X_train, y_train)
    training_time = time.time() - start_time
    return tree_model, training_time

# K-Nearest Neighbors Classifier with daal4py (GPU enabled)
def train_knn_model_gpu(X_train, y_train):
    start_time = time.time()
    with dpctl.SyclQueue("level_zero:gpu") as queue:
        knn_algo = d4p.kdtree_knn_classification_training(nClasses=3, k=3)
        knn_model = knn_algo.compute(X_train, y_train)
    training_time = time.time() - start_time
    return knn_model, training_time

# Support Vector Machine (SVM) Classifier with daal4py (GPU enabled)
def train_svm_model_gpu(X_train, y_train):
    start_time = time.time()
    with dpctl.SyclQueue("level_zero:gpu") as queue:
        svm_algo = d4p.svm_training(nClasses=3)
        svm_model = svm_algo.compute(X_train, y_train)
    training_time = time.time() - start_time
    return svm_model, training_time



In [5]:
# 3. Use Dask delayed to parallelize the training of models
svm_task_delayed = dask.delayed(train_svm_model_gpu)(X_train, y_train)
knn_task_delayed = dask.delayed(train_knn_model_gpu)(X_train, y_train)
tree_task_delayed = dask.delayed(train_tree_model_gpu)(X_train, y_train)

# 4. Compute the tasks in parallel
trained_models = dask.compute(svm_task_delayed, knn_task_delayed, tree_task_delayed, scheduler='threads')

# 5. Unpack the trained models and their training times
(svm_model, svm_training_time), (knn_model, knn_training_time), (tree_model, tree_training_time) = trained_models

# 6. Print training times for each model
print(f"SVM Training Time (GPU): {svm_training_time:.4f} seconds")
print(f"KNN Training Time (GPU): {knn_training_time:.4f} seconds")
print(f"Decision Tree Training Time (GPU): {tree_training_time:.4f} seconds")


SyclQueueCreationError: SYCL Device 'level_zero:gpu' could not be created.